In [ ]:
%matplotlib inline
import copy, datetime, os, re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas
import seaborn as sns
from scipy import stats
import data

In [ ]:
results_path = 'results'

ids = ['{}_{}'.format(*p) for p in data.get_pairs()]

groups = [
    'df_cpu',
    'df_gpu',
    'ants'
]

structures = {}
structures['All'] = { 'name': 'All', 'brief': 'All' }

with open(os.path.join('structures.csv'), 'r') as f:
    l = f.readline() # Header
    for l in f:
        t = l.split(',')
        structures[t[0]] = { 'name': t[1], 'brief': t[2].strip() }

# Combine left and right
        
structures_combined = {}
for k, s in structures.items():
    brief = s['brief']
    name = s['name']
    if name[-1:] in ['L', 'R']:
        brief = brief[:-1]
        name = name[:-2]
    structures_combined[k] = { 'name': name, 'brief': brief }

In [ ]:
# Load dice data

group_dice_data = []
for g in groups:
    pair_dice_data = []
    for p in ids:
        d = pandas.read_csv(os.path.join(results_path, g, '{}.csv'.format(p)))[['Label','Dice']]
        pair_dice_data.append(d)
    group_dice_data.append(pandas.concat(pair_dice_data, keys=ids, names=['pair']))
    
dice_data = pandas.concat(group_dice_data, keys=groups, names=['group'])
dice_data = dice_data.reset_index()
dice_data = dice_data.drop(columns='level_2')

dice_data['LabelName_All'] = dice_data['Label'].map(lambda x : structures[x]['brief'])
dice_data['LabelName'] = dice_data['Label'].map(lambda x : structures_combined[x]['brief'])

# Pretty group names
group_map = {
    'df_cpu': 'deform (CPU)',
    'df_gpu': 'deform (GPU)',
    'ants': 'ANTs'
}
dice_data['Method'] = dice_data['group'].map(lambda x : group_map[x])
dice_data.groupby('Method').Dice.describe()

In [ ]:
# Load run time

def to_datetime(s):
    return datetime.datetime.strptime(s, '%H:%M:%S.%f')

group_time_data = []
for g in groups:
    pair_time_data = []
    for p in ids:
        with open(os.path.join(results_path, g, '{}_log.txt'.format(p))) as f:
            lines = f.readlines()

        d = {}

        for i in range(0,5):
            d['level_{}'.format(i)] = None

        match = re.match('INF \d{2}-\d{2} (\d+:\d+:\d+.\d+) \|.*$', lines[0])
        if match:
            # deform
            start = to_datetime(match.group(1))
            match = re.match('INF \d{2}-\d{2} (\d+:\d+:\d+.\d+) \|.*$', lines[-4])
            stop = to_datetime(match.group(1))

            d['total'] = (stop - start).seconds

            current_level = None
            current_level_start = None

            for l in lines:
                m = re.match('INF \d{2}-\d{2} (\d+:\d+:\d+.\d+) \| Performing registration level (\d)+$', l)
                if m:
                    current_level_start = to_datetime(m.group(1))
                    current_level = m.group(2)
                m = re.match('INF \d{2}-\d{2} (\d+:\d+:\d+.\d+) \| Energy: \d+\.*\d*, Iterations: \d+$', l)
                if m:
                    stop = to_datetime(m.group(1))
                    d['level{}'.format(current_level)] = (stop - current_level_start).seconds
                    current_level = None
                    current_level_start = None
        else:
            # ANTs
            # Only time the SyN stage

            d['total'] = None
            for l in lines:
                m = re.match('\s*Elapsed time \(stage 2\): (\d+\.*\d*$)', l)
                if m:
                    d['total'] = float(m.group(1))
                    break
            
        
        df = pandas.DataFrame([d], columns=['total'] + ['level{}'.format(l) for l in range(0,5)])
        pair_time_data.append(df)
    group_time_data.append(pandas.concat(pair_time_data, keys=ids, names=['pair']))
time_data = pandas.concat(group_time_data, keys=groups, names=['group'])
time_data = time_data.reset_index()
time_data = time_data.drop(columns='level_2')
time_data['Method'] = time_data['group'].map(lambda x : group_map[x])
time_data.groupby('Method').total.describe()

In [ ]:
print(dice_data[(dice_data.group=='df_gpu')].describe())
print(dice_data[(dice_data.group=='ants')].describe())
t, p = stats.ttest_rel(dice_data[(dice_data.group=='df_gpu')].Dice, dice_data[(dice_data.group=='ants')].Dice)
print('t-test: p={}'.format(p))

In [ ]:
# Sort region dice in boxplot by median

medians = dice_data[dice_data.group=='df_gpu'].groupby('LabelName').median()
medians = medians.sort_values('Dice', ascending=False)

dice_data_median = dice_data.copy()
dice_data_median.LabelName = dice_data_median.LabelName.astype('category')
dice_data_median.LabelName.cat.set_categories(list(medians.index), inplace=True)
dice_data_median = dice_data_median.sort_values(['LabelName', 'Method'], ascending=[True, False])
dice_data_median = dice_data_median.reset_index()

sns.set(font_scale=1.5)
sns.set_style("white")

fig=plt.figure(figsize=(15, 4.5), dpi=300, facecolor='w', edgecolor='k')
ax = fig.add_axes()

ax = sns.boxplot(data=dice_data_median[dice_data_median.group.isin(['df_gpu','ants'])], y='Dice', x='LabelName', hue='Method', ax=ax)
ax.set(xlabel='')
_ = plt.xticks(rotation=90)
fig.tight_layout()
fig.savefig('figures/results_dice_box.pdf', pad_inches=0, bbox_inches='tight')